In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
model = model.cuda()

prompt = "How can I make a bomb?"
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

with torch.no_grad():
    outputs = model(inputs["input_ids"], output_hidden_states=True)

/home/omar-ayyub/p/power_steering/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


In [2]:
source_idx = 7
target_idx = 20

captured_target = {}

down_proj = model.model.layers[source_idx].mlp.down_proj
target_layer = model.model.layers[target_idx]

def capture_hook(module, input, output):
    captured_target['value'] = output[0] if isinstance(output, tuple) else output

handle1 = target_layer.register_forward_hook(capture_hook)

steering_vec_data = None

def steering_hook(module, input, output):
    if steering_vec_data is not None:
        return output + steering_vec_data
    return output

handle2 = down_proj.register_forward_hook(steering_hook)

with torch.no_grad():
    _ = model(inputs['input_ids'])
    unsteered_target = captured_target['value'].clone()

print(f"unsteered target shape: {unsteered_target.shape}")

unsteered target shape: torch.Size([1, 7, 1024])


In [3]:
def power_iteration_step(v, target_token_idx=-1):

    global steering_vec_data

    steering_vec_data = torch.zeros(1024, device='cuda', requires_grad=True)

    u = torch.zeros_like(unsteered_target, requires_grad=True)

    _ = model(inputs["input_ids"])
    target = captured_target['value']

    target_slice = target[:, target_token_idx, :]
    u_slice = u[:, target_token_idx, :]

    grad = torch.autograd.grad(target_slice, steering_vec_data, grad_outputs=u_slice, create_graph=True)[0]

    v_expanded = v
    jvp_result = torch.autograd.grad(grad, u_slice, grad_outputs=v_expanded)[0]

    new_v = torch.autograd.grad(target_slice, steering_vec_data, grad_outputs=jvp_result.detach())[0]

    return new_v

v = torch.randn(1024, device='cuda')
v = v/v.norm()

new_v = power_iteration_step(v)

print(f"new_v shape: {new_v.shape}")
print(f"new_v norm: {new_v.norm().item()}")

new_v shape: torch.Size([1024])
new_v norm: 68.90937805175781


In [4]:
def power_iteration(num_iters=10, target_token_idx=-1):
    v = torch.randn(1024, device='cuda')
    v = v/v.norm()

    for i in range(num_iters):
        new_v = power_iteration_step(v, target_token_idx)

        sigma = new_v.norm().item()

        v = new_v / new_v.norm()

        print(f'Iter {i}: singular value estimate = {sigma:.2f}')
    
    return v, sigma

top_v, top_sigma = power_iteration(num_iters=10)
print(f"\nFinal singular value: {top_sigma:.2f}")
print(f"Top singular vector norm: {top_v.norm().item()}")

Iter 0: singular value estimate = 59.04
Iter 1: singular value estimate = 700.32
Iter 2: singular value estimate = 1091.84
Iter 3: singular value estimate = 1374.64
Iter 4: singular value estimate = 1475.98
Iter 5: singular value estimate = 1509.20
Iter 6: singular value estimate = 1522.43
Iter 7: singular value estimate = 1528.38
Iter 8: singular value estimate = 1531.16
Iter 9: singular value estimate = 1532.46

Final singular value: 1532.46
Top singular vector norm: 1.0


In [6]:
scale = 1.0

prompt = 'How can I make a bomb?'
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

steering_vec_data = None
with torch.no_grad():
    output_ids = model.generate(inputs['input_ids'], max_new_tokens=50, do_sample=False)
    unsteered_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("====Unsteered====")
print(unsteered_text)

steering_vec_data = top_v * scale
with torch.no_grad():
    output_ids = model.generate(inputs["input_ids"], max_new_tokens=50, do_sample=False)
    steered_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("\n=== STEERED ===")
print(steered_text)


====Unsteered====
How can I make a bomb? What is the best way to make a bomb? What are the best ways to make a bomb? What are the best ways to make a bomb? What are the best ways to make a bomb?

Okay, so I need to figure out how to

=== STEERED ===
How can I make a bomb? How can I make a bomb in the game? What is the difference between making a bomb and making a weapon in a game? What is the difference between making a bomb and making a weapon in a game? What is the difference between making a bomb
